In [45]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [46]:
class Vertice:
    
    def __init__(self, valor, direcionado=True):
        self.__valor = valor
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getValor(self):
        return self.__valor
    
    def setValor(self, valor):
        self.__valor = valor
        
    def getArestas(self):
        return self.__arestas
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida())+ len(self.getArestasEntrada())
    
    def getAdjacentes(self, v):
        listaVerticesAdjacentes = []
        for arestas_de_saida in v.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getvDestino())
        return listaVerticesAdjacentes

In [47]:
class Aresta:
    def __init__(self, vOrigem, vDestino, peso, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getValor(self):
        return self.__peso

In [48]:
from collections import deque
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getVerticeByValor(self, valor):
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None
    
    def getArestas(self):
        return self.__arestas
    
    def checkHandShakingLemma(self):
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):
        if v not in visitados: # se v nao foi visitado
            visitados.append(v) # marca vertice como visitado
        if len(v.getAdjacentes(v)) == 0: # vertice escolhido nao tem adjacentes
            self.dfs(graph, next(iter(graph.getVertices())), visitados) # chamada recursiva pegando o proximo vertice do set
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(v): #percorre todos os adjacentes a ele
                if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                    self.dfs(graph, adjacente, visitados) # chamada recursiva para cada adjacente
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):
        fila.append(v)  # adiciona o vertice v a fila
        if v not in visitados:  # se vertice v nao esta em visitados
            visitados.append(v)  # adiciona vertice v a visitados
        while fila:  # enquanto houver vertices na fila
            vertice = fila.popleft()  # tira vertice ja visitado da fila
            if len(vertice.getArestasSaida()) == 0: # vertice escolhido nao tem adjacentes
                self.bfs(next(iter(self.getVertices())), visitados, fila) # chamada recursiva pegando o proximo vertice do set   
            else:
                for adjacente in vertice.getAdjacentes(v): #percorre todos os adjacentes a ele
                    if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                        visitados.append(adjacente)  # insere o adjacente em visitados
                        self.bfs(adjacente, visitados, fila) # chamada recursiva pegando o proximo vertice do set
        return visitados  # retorna a lista de visitados  
    
    def bfsByValue(self, value):
        verticeInicial = next(iter(self.getVertices()))
        vistadosValor = []
        for vertice in self.bfs(verticeInicial):
            vistadosValor.append(vertice.getValor())
        if value in vistadosValor:
            return True
        return False
    def ehEuler(self):
        todosPar = True
        grauImpar = []
        for vertice in self.getVertices():
            if vertice.getGrau() % 2 != 0:
                todosPar = False
                grauImpar.append(vertice)
        return todosPar or len(grauImpar) == 2
    
    def inserirVertice(self, valor):
        if(self.bfsByValue(valor)):
            return False
        self.__vertices.add(Vertice(valor))
        return True
    
    def inserirAresta(self,origem, destino, peso):
        vOrigem = self.bfsByValue(origem)
        vDestino = self.bfsByValue(destino)
        if vOrigem and vDestino:
            self.__arestas.add(Aresta(self.getVerticeByValor(origem), self.getVerticeByValor(destino), peso))
            return True
        return False
    
    def removerAresta(self, peso, origem, destino):
        vOrigem = self.getVerticeByValor(origem)
        vDestino = self.getVerticeByValor(destino)
        if (vDestino or vOrigem) != None:
            for aresta in self.getArestas():
                if aresta.getvOrigem() == vOrigem and aresta.getvDestino() and aresta.getValor() == peso:
                    self.__arestas.remove(aresta)
                    return True
        return False
    
    def removerVertice(self, valor):
        vertice = self.getVerticeByValor(valor)
        arestasRemovidas = []
        if vertice in self.__vertices:
            for aresta in self.getArestas():
                if aresta.getvOrigem() == vertice or aresta.getvDestino() == vertice:
                    arestasRemovidas.append(aresta)
            for aresta in arestasRemovidas:
                self.__arestas.remove(aresta)
            self.__vertices.remove(vertice)
            return True
        return False
        
        
    
    
            
        
            

In [49]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True )

In [50]:
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5})
G.setArestas({a1, a2, a3, a4, a5})

In [51]:
for v in G.getVertices():
    print(v.getValor(), end="\t")

2	1	5	3	4	

In [52]:
for a in G.getArestas():
    print(a.getvOrigem().getValor(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getValor(), end="\t")

4 ---> 1	2 ---> 3	4 ---> 5	3 ---> 4	1 ---> 2	

In [53]:
v1.getGrau()

2

In [54]:
G.checkHandShakingLemma()

True

#### Testando busca em profundidade

In [55]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em profundidade, iniciando com o vértice {vertice.getValor()}:")
    for v in G.dfs(G, vertice, visitados=[]):
        print(str(v.getValor())+"\t", end="")
    print("\n.................................................")  

Busca em profundidade, iniciando com o vértice 3:
3	4	1	2	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 5:
5	3	4	1	2	6	
.................................................
Busca em profundidade, iniciando com o vértice 6:
6	3	4	1	2	5	
.................................................
Busca em profundidade, iniciando com o vértice 4:
4	1	2	3	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 1:
1	2	3	4	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 2:
2	3	4	1	6	5	
.................................................


In [56]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em largura, iniciando com o vértice {vertice.getValor()}:")
    for v in  G.bfs(vertice, visitados = [], fila = deque([])):
        print(str(v.getValor())+"\t", end="")
    print("\n............................................")   

Busca em largura, iniciando com o vértice 1:
1	2	3	4	5	6	
............................................
Busca em largura, iniciando com o vértice 3:
3	4	1	2	5	6	
............................................
Busca em largura, iniciando com o vértice 5:
5	1	2	3	4	6	
............................................
Busca em largura, iniciando com o vértice 4:
4	1	2	3	5	6	
............................................
Busca em largura, iniciando com o vértice 2:
2	3	4	1	5	6	
............................................
Busca em largura, iniciando com o vértice 6:
6	1	2	3	4	5	
............................................


In [57]:
G.bfsByValue(1)

True

In [58]:
G.ehEuler()

True

In [59]:
G.getVertices()

{<__main__.Vertice at 0x28cd30bc340>,
 <__main__.Vertice at 0x28cd30bca90>}

In [60]:
G.inserirVertice(1)

False

In [61]:
for v in G.getVertices():
    print(v.getValor())

1
3
5
4
2
6


In [62]:
G.inserirVertice(11)

True

In [63]:
for v in G.getVertices():
    print(v.getValor())

1
3
5
4
11
2
6


In [64]:
G.getArestas()

{<__main__.Aresta at 0x28cd16071f0>,
 <__main__.Aresta at 0x28cd30bca00>}

In [65]:
G.inserirAresta(2, 3, 15)

True

In [66]:
G.getArestas()

{<__main__.Aresta at 0x28cd16071f0>,
 <__main__.Aresta at 0x28cd314ff40>}

In [67]:
G.removerAresta(60, 4, 6)

True

In [68]:
G.getArestas()

{<__main__.Aresta at 0x28cd16076a0>,
 <__main__.Aresta at 0x28cd314ff40>}

In [69]:
G.removerVertice(1)

True

In [70]:
G.getVertices()

{<__main__.Vertice at 0x28cd30bc340>,
 <__main__.Vertice at 0x28cd314ff70>}

In [71]:
G.getArestas()

{<__main__.Aresta at 0x28cd16076a0>,
 <__main__.Aresta at 0x28cd314ff40>}

In [72]:
G.removerVertice(11)

True

In [73]:
G.getVertices()

{<__main__.Vertice at 0x28cd30bc340>,
 <__main__.Vertice at 0x28cd30bca90>}